In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns 
pd.set_option('display.max_columns', None)

In [2]:
data = pd.read_csv("properati.csv")

In [3]:
data.shape

(121220, 26)

In [4]:
# borro los duplicados son 6 mil
data1 = data.drop_duplicates(subset=["property_type","place_name","geonames_id","lat-lon","price","surface_total_in_m2","surface_covered_in_m2","description"],keep="first")
data1.shape,data.shape

((115151, 26), (121220, 26))

In [5]:
desc_dupl = data.duplicated(subset=["property_type","place_name","geonames_id","lat-lon","price","surface_total_in_m2","surface_covered_in_m2","description"])
sindesc_dupl = data.duplicated(subset=["property_type","place_name","geonames_id","lat-lon","price","surface_total_in_m2","surface_covered_in_m2"])
sindesc_dupl1 = data1.duplicated(subset=["property_type","place_name","geonames_id","lat-lon","price","surface_total_in_m2","surface_covered_in_m2"],keep = False)
desc_dupl.sum(),sindesc_dupl.sum(),sindesc_dupl1.sum()
# pd.set_option('display.max_colwidth', None)
# des= data1.duplicated(subset=["description"])
# data1[sindesc_dupl1].head()

(6069, 13873, 12560)

In [6]:
#me fijo si los nulos son iguales en las columnas de precios. 
(data1.price.isnull() == data1.price_aprox_usd.isnull()).sum(),data1.shape[0]
(data1.price.isnull() == data1.price_aprox_local_currency.isnull()).sum(),data1.shape[0]

(115151, 115151)

In [7]:
# Me fijo si los nulos estan en la misma fila en ambas columnas
set(data1.price.isnull().index).issubset(set(data1.price_aprox_usd.isnull().index))
set(data1.price.isnull().index).issubset(set(data1.price_aprox_local_currency.isnull().index))

True

In [9]:
# quiero ver de donde sale el precio por metro cuadrado 
# preciom2 = data.price_aprox_local_currency / data.surface_total_in_m2
# preciom2usd = data.price_aprox_usd / data.surface_total_in_m2
# preciototal = data.price / data.surface_total_in_m2
# preciototal_cov = data.price / data.surface_covered_in_m2
# (preciom2 == data.price_per_m2).sum(),(preciom2usd == data.price_usd_per_m2).sum(),(preciototal == data.price_usd_per_m2).sum(),(preciototal == data.price_per_m2).sum(),(preciototal_cov == data.price_per_m2).sum()
# # preciom2cov = data.price_aprox_local_currency / data.price_per_m2
# # veo el total de resultados que obtuve 
# (preciototal == data.price_usd_per_m2).sum() + (preciototal == data.price_per_m2).sum() + (preciototal_cov == data.price_per_m2).sum(),

In [10]:
# veo la cantidad que hay 
# data.price_usd_per_m2.dropna().size,data.price_usd_per_m2.size

## empiezo a usar los regex para completar los precios en dolares 

In [8]:
import re

In [9]:
data1.price_aprox_usd.notnull().sum(),data1.price_aprox_usd.isnull().sum()

(97882, 17269)

Floor

In [10]:
m_pisos = data1.floor.notnull()
pisos = data1.loc[m_pisos, :]
pisos.floor.count()

7462

In [11]:
data1.floor.isnull()

0         True
1         True
2         True
3         True
4         True
          ... 
121215    True
121216    True
121217    True
121218    True
121219    True
Name: floor, Length: 115151, dtype: bool

In [12]:
m_tipo_propiedad = data1.property_type == ('apartment')
tipo_propiedad = data1.loc[m_tipo_propiedad,:]

m_pisos = tipo_propiedad.floor.isnull()
pisos_nulos = tipo_propiedad.loc[m_pisos, :]

In [24]:
# estilo #ºer piso

# busco pisos en titulo
pat_pisos = r"(?P<pisos>\d?\d)(([^A-z]{1,2}?)|([A-z]{2,3})?(\s?))([pP][iI][sS][oO])"
regexs_piso = re.compile(pat_pisos)
pisos_tit = pisos_nulos.title.apply(lambda x: regexs_piso.search(str(x)))
pisos_limpio = pisos_tit[pisos_tit.notnull()].apply(lambda x: x.group('pisos')).copy()
data1.loc[pisos_limpio.index,'floor'] = pisos_limpio.astype(float)

m_pisos2 = tipo_propiedad.floor.isnull()
pisos_nulos2 = tipo_propiedad.loc[m_pisos2, :]

# busco pisos en description
pat_pisos = r"(?P<pisos>\d?\d)(([^A-z]{1,2}?)|([A-z]{2,3})?(\s?))([pP][iI][sS][oO])"
regexs_piso = re.compile(pat_pisos)
pisos_desc = pisos_nulos2.description.apply(lambda x: regexs_piso.search(str(x)))
pisos_limpio_2 = pisos_desc[pisos_desc.notnull()].apply(lambda x: x.group('pisos')).copy()
data1.loc[pisos_limpio_2.index,'floor'] = pisos_limpio_2.astype(float)


In [25]:
m_pisos = tipo_propiedad.floor.isnull()
pisos_nulos = tipo_propiedad.loc[m_pisos, :]

#estilo piso ##

# busco pisos en titulo 
pat_pisos = r"([pP][iI][sS][oO])(\s?)(?P<pisos>\d?\d)"
regexs_piso = re.compile(pat_pisos)
pisos_tit = pisos_nulos.title.apply(lambda x: regexs_piso.search(str(x)))
pisos_limpio = pisos_tit[pisos_tit.notnull()].apply(lambda x: x.group('pisos')).copy()
data1.loc[pisos_limpio.index,'floor'] = pisos_limpio.astype(float)

m_pisos2 = tipo_propiedad.floor.isnull()
pisos_nulos2 = tipo_propiedad.loc[m_pisos2, :]

# busco pisos en description
pat_pisos = r"([pP][iI][sS][oO])(\s?)(?P<pisos>\d?\d)"
regexs_piso = re.compile(pat_pisos)
pisos_desc = pisos_nulos2.description.apply(lambda x: regexs_piso.search(str(x)))
pisos_limpio_2 = pisos_desc[pisos_desc.notnull()].apply(lambda x: x.group('pisos')).copy()
data1.loc[pisos_limpio_2.index,'floor'] = pisos_limpio_2.astype(float)

In [26]:
m_pisos = tipo_propiedad.floor.isnull()
pisos_nulos = tipo_propiedad.loc[m_pisos, :]
#estilo PB 

# busco pisos en titulo 
pat_pisos = r"(?P<pisos>[pP][bB])"
regexs_piso = re.compile(pat_pisos)
pisos_tit = pisos_nulos.title.apply(lambda x: regexs_piso.search(str(x)))
pisos_limpio = pisos_tit[pisos_tit.notnull()].apply(lambda x: x.group('pisos')).copy()
data1.loc[pisos_limpio.index,'floor'] = pisos_limpio

m_pisos2 = tipo_propiedad.floor.isnull()
pisos_nulos2 = tipo_propiedad.loc[m_pisos2, :]

# busco pisos en description
pat_pisos = r"(?P<pisos>[pP][bB])"
regexs_piso = re.compile(pat_pisos)
pisos_desc = pisos_nulos2.description.apply(lambda x: regexs_piso.search(str(x)))
pisos_limpio_2 = pisos_desc[pisos_desc.notnull()].apply(lambda x: x.group('pisos')).copy()
data1.loc[pisos_limpio_2.index,'floor'] = pisos_limpio_2

In [27]:
print(pisos_limpio.count().sum())
print(pisos_limpio_2.count().sum())

270
3172


In [28]:
tipo_propiedad.floor.isnull().sum()

59539

In [29]:
m_tipo_propiedad2 = data1.property_type == ('apartment')
tipo_propiedad2 = data1.loc[m_tipo_propiedad2,:]
tipo_propiedad2.floor.isnull().sum()

45029

In [30]:
data1_grouped = data1.groupby('property_type')
data1_grouped.size()

property_type
PH            5597
apartment    65842
house        39653
store         4059
dtype: int64

In [31]:
#Hago un regex para saber si tiene cochera  
pat_cochera = r'(?P<cochera>([Cc]ochera)|([Gg]ara[gj]e)s?)'
regex_cochera = re.compile(pat_cochera)
cochera1= data1.description.apply(lambda x: regex_cochera.search(str(x)))
cochera1

0                                                      None
1         <re.Match object; span=(333, 340), match='coch...
2                                                      None
3                                                      None
4                                                      None
                                ...                        
121215    <re.Match object; span=(202, 209), match='Coch...
121216    <re.Match object; span=(749, 755), match='Gara...
121217                                                 None
121218                                                 None
121219                                                 None
Name: description, Length: 115151, dtype: object

In [32]:
# creo la columna cochera, asignandole 1 si tiene y 0 sino tiene 
data1.loc[cochera1.notnull(),'cochera'] = 1
data1.loc[cochera1.isnull(),'cochera'] = 0
data1.cochera

0         0.0
1         1.0
2         0.0
3         0.0
4         0.0
         ... 
121215    1.0
121216    1.0
121217    0.0
121218    0.0
121219    0.0
Name: cochera, Length: 115151, dtype: float64

In [33]:
data1.cochera.value_counts()

0.0    73295
1.0    41856
Name: cochera, dtype: int64

In [34]:
mask_geo_place_null = data.geonames_id.isnull()
df_geoname_null = data.loc[mask_geo_place_null,:]
df_geoname_null.isnull().sum()

Unnamed: 0                        0
operation                         0
property_type                     0
place_name                       23
place_with_parent_names           0
country_name                      0
state_name                        0
geonames_id                   18717
lat-lon                        8185
lat                            8185
lon                            8185
price                          1765
currency                       1765
price_aprox_local_currency     1765
price_aprox_usd                1765
surface_total_in_m2            5499
surface_covered_in_m2          2053
price_usd_per_m2               7021
price_per_m2                   3567
floor                         17911
rooms                         12166
expenses                      16407
properati_url                     0
description                       0
title                             0
image_thumbnail                 395
dtype: int64